In [54]:
import pandas as pd
import asyncio
import httpx
from bs4 import BeautifulSoup
import json

df = pd.read_excel("prices3.xlsx")





In [55]:
HEADERS = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en",
    "content-type": "application/json",
    "origin": "https://www.boutiqaat.com",
    "param": '{"gender":"4194","app_version":"8.0.0","platform":"web","deviceId":"1759937968632-266715","exp_list":{"exp-55139":"default","exp-36818":"disabled","exp-36740":"default"}}',
    "priority": "u=1, i",
    "referer": "https://www.boutiqaat.com/",
    "sec-ch-ua": '"Google Chrome";v="141", "Not?A_Brand";v="8", "Chromium";v="141"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Linux"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"
}


async def fetch_slug(client, sku):
    url = "https://ksa-api.boutiqaat.com/searchplus/rest/V2/global/suggest"
    payload = {
        "productId": "",
        "tvId": "",
        "slug": "",
        "celebrityId": "",
        "categoryId": "",
        "brandId": "",
        "celebrityIds": "",
        "categoryIds": "",
        "brandIds": "",
        "optionId": "",
        "attributeId": "",
        "countryCodeAndLanguage": "kw_en",
        "numberOfRecords": 20,
        "newOrAll": "",
        "featuredOnly": None,
        "sortKey": "",
        "sortDirection": "",
        "searchString": sku
    }
    resp = await client.post(url, headers=HEADERS, json=payload, timeout=30)
    data = resp.json()
    try:
        return data["data"][0]["data"][0]["slug"]
    except (IndexError, KeyError):
        return None




In [56]:

async def fetch_final_price(client, slug):
    if not slug:
        return None
    url = f"https://www.boutiqaat.com/en-kw/men/{slug}"
    resp = await client.get(url, timeout=30)
    soup = BeautifulSoup(resp.text, "html.parser")
    script_tag = soup.find("script", {"id": "__NEXT_DATA__"})
    if not script_tag:
        return None
    data = json.loads(script_tag.string)
    try:
        return data["props"]["pageProps"]["response"][0]["final_price_with_tax"]
    except (KeyError, IndexError):
        return None




In [57]:
async def fetch_price_for_sku(client, sku):
    slug = await fetch_slug(client, sku)
    price = await fetch_final_price(client, slug)
    return sku, price



In [58]:
async def main():
    async with httpx.AsyncClient() as client:
        tasks = [fetch_price_for_sku(client, sku) for sku in df["sku"]]
        results = await asyncio.gather(*tasks)

    # Add results to dataframe
    df["web_price"] = [price for sku, price in results]
    df["match"] = df.apply(lambda row: "match" if float(row["web_price"] or 0) == float(row["special price"]) else "item mismatch", axis=1)
    print(df["match"])
    df.to_excel("comparison2_results.xlsx", index=False)
    print("Comparison done, saved to comparison_results.xlsx")


await main()


0             match
1     item mismatch
2             match
3     item mismatch
4             match
5     item mismatch
6             match
7     item mismatch
8             match
9     item mismatch
10            match
11    item mismatch
12            match
13    item mismatch
14            match
15    item mismatch
16            match
17    item mismatch
18            match
19    item mismatch
20            match
21    item mismatch
22            match
23    item mismatch
24            match
25    item mismatch
26            match
27    item mismatch
28            match
29    item mismatch
30            match
31    item mismatch
32            match
33    item mismatch
34            match
35    item mismatch
36            match
37    item mismatch
38            match
39    item mismatch
40            match
41    item mismatch
42            match
43    item mismatch
44            match
45    item mismatch
46            match
47    item mismatch
48            match
49    item mismatch
